<img src="image.jpg">

 # PyPRT - Building Modeling Optimization

This notebook is used to show how the Python bindings for PRT can be employed in optimizing the rule attributes of a generated model to achive predefined performance goals.

### Import PyPRT module and define paths-related functions

In [1]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "..", "src"))
from utility import visualize_PRT_results, combine_reports, summarize_matrix, summarize_report

SDK_PATH = os.path.join(os.getcwd(), "..", "install", "bin")
sys.path.append(SDK_PATH)

import pyprt

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import optimize

In [3]:
CS_FOLDER = os.getcwd()
def asset_file(filename):
    return os.path.join(CS_FOLDER, filename)

In [4]:
print("\nInitializing PRT.")
pyprt.initialize_prt(SDK_PATH)

if not pyprt.is_prt_initialized():
    raise Exception("PRT is not initialized")


Initializing PRT.


## Optimization Problem Definition

In this notebook, we are using a CGA rule that allows us to generate modern and sustainable buildings. The architecture of these buildings is focused on the green aspect of the building. Therefore, in addition to be modern and esthetical, the generated building answers also to the actual requirements of the city which are to encourage the integration of greenery on the buildings.

<img src="green_building.png">

### The variables

Among the values reported by the CGA rule, we want to optimize the __height of the generated building__. In this example, we would like the building height to be as close as possible to __10 meters__.
The next step is to define the inputs that will vary and on which we will optimize. These inputs must be CGA rule attributes. Here we will take the building height and the k factor as optimization inputs.

In [23]:
goal_str = "Building Height" # correspond to a CGA reported value

input_attr = ["buildingHeight:float","k:float"]
x0 = [13, 0.1] # input attributes initial value

In [43]:
shape_geo = asset_file("greenbuildingfootprint_0.obj") # parcel/geometry on which the model will be generated
mod = pyprt.ModelGenerator(shape_geo)

In [44]:
rpk = asset_file("simple_rule2.rpk")
rule_file_attr = "ruleFile:string=bin/simple_rule2.cgb"
start_rule_attr = "startRule:string=Default$Footprint"

In [45]:
def fct(x): # function to minimize
    input_val0 = input_attr[0]+"="+str(x[0])
    input_val1 = input_attr[1]+"="+str(x[1])
    
    generated_mod = mod.generate_model(rpk, [rule_file_attr, start_rule_attr, input_val0, input_val1])
    
    if len(generated_mod) == 1:
        rep = summarize_report(combine_reports(generated_mod[0]))
        #return rep[goal_str] # output to optimize : y
        return ((-10+rep[goal_str])*(-10+rep[goal_str]))

    else:
        print("Error in optimization process.")  

## Optimization Problem Resolution

In [46]:
bounds = [(0,25),(0,0.6)] # sequence of (min,max)

In [47]:
res = optimize.differential_evolution(fct, bounds)
res

     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 1053
     nit: 34
 success: True
       x: array([9.99999978, 0.47106757])

### Conclusion

The set of attributes that minimizes the output is: __building height of 10 meters__ and __factor k of 0.39__.

##### Optimization with envelope.rpk --> too slow...

In [56]:
goal_str_green = "green area" # correspond to a CGA reported value

input_attr_green = ["lot_coverage_parameter:float"]
x0_green = [10]

In [57]:
rpk_green = asset_file("envelope1806.rpk")
rule_file_attr_green = "ruleFile:string=rules/typology/envelope.cgb"
start_rule_attr_green = "startRule:string=Default$Lot"

In [63]:
def fct_green(x):
    input_val0 = input_attr_green[0]+"="+str(x[0])
    generated_mod = mod.generate_model(rpk_green, [rule_file_attr_green, start_rule_attr_green, input_val0])
    
    if len(generated_mod) == 1:
        rep = summarize_report(combine_reports(generated_mod[0]))
        return -rep[goal_str_green]
    else:
        print("Error in optimization process.")

In [64]:
bounds_green = [(0,100)]

Test

In [65]:
fct_green(x0_green)

-1239.515014410019

res_green = optimize.differential_evolution(fct_green, bounds_green)
res_green

In [68]:
print("\nShutdown PRT.")
pyprt.shutdown_prt()


Shutdown PRT.
